<a href="https://colab.research.google.com/github/hyung6370/Univ.AI/blob/main/%EA%B9%80%ED%98%95%EC%A4%80_%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EB%AC%B8%EC%A0%9C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import numpy as np
from datetime import datetime      # datetime.now() 를 이용하여 학습 경과 시간 측정

def sigmoid(x):         # sigmoid 함수
    return 1 / (1+np.exp(-x))

def cross_entropy(t, y) :
    delta = 1e-7    # log 무한대 발산 방지
    return -np.sum(t*np.log(y+delta) + (1-t)*np.log((1-y)+delta))    

def numerical_derivative(f, x):      # 수치미분 함수
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
    return grad

class LogicGate:
    def __init__(self, gate_name, xdata, tdata):
        self.name = gate_name
        # 입력 데이터, 정답 데이터 초기화
        self.xdata = xdata.reshape(4,2)  # 4개의 입력데이터 x1, x2 에 대하여 batch 처리 행렬
        self.tdata = tdata.reshape(4,1)  # 4개의 입력데이터 x1, x2 에 대한 각각의 계산 값 행렬
        
        # 2층 hidden layer unit : 3개 가정,  가중치 W2, 바이어스 b2 초기화
        self.W2 = np.random.rand(2,3)  # weight, 2 X 3 matrix
        self.b2 = np.random.rand(3)
        
        # 3층 hidden layer unit : 2 개 , 가중치 W3, 바이어스 b3 초기화
        self.W3 = np.random.rand(3,2)  # weight, 3 X 2 matrix
        self.b3 = np.random.rand(2)

        # 4층 output layer unit : 1 개 , 가중치 W4, 바이어스 b4 초기화
        self.W4 = np.random.rand(2,1)  # weight, 2 X 1 matrix
        self.b4 = np.random.rand(1)
                        
        # 학습률 learning rate 초기화
        self.lr = 1e-2
        print(self.name + " object is created")
            
    def feed_forward(self):        # errFunc()함수 대신 feed forward를 통하여 손실함수(cross-entropy) 값 계산
        z2 = np.dot(self.xdata, self.W2) + self.b2  # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                            # 은닉층의 출력
        z3 = np.dot(a2, self.W3) + self.b3          # 은닉층의 선형회귀 값
        a3 = sigmoid(z3)                            # 은닉층의 출력
        z4 = np.dot(a3, self.W4) + self.b4          # 출력층의 선형회귀 값
        y = a4 = sigmoid(z4)                        # 출력층의 출력
        return cross_entropy(self.tdata, y)         # 출력의 손실값 리턴
    
    def errValue(self):             # 외부 출력을 위한 손실함수(cross-entropy) 값 계산 
        return  self.feed_forward()
    
    def train(self):            # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
        f = lambda x : self.feed_forward()
        start = datetime.now()
        for step in range(55001):
            self.W2 -= self.lr * numerical_derivative(f, self.W2)
            self.b2 -= self.lr * numerical_derivative(f, self.b2)
            self.W3 -= self.lr * numerical_derivative(f, self.W3)
            self.b3 -= self.lr * numerical_derivative(f, self.b3)
            self.W4 -= self.lr * numerical_derivative(f, self.W4)
            self.b4 -= self.lr * numerical_derivative(f, self.b4)
            if (step % 2000 == 0):
                print("Step = {:<5d}\tError Val = {:.4f}".format(step, self.errValue()))
        print("Training time = ", datetime.now() - start)

    def predict(self, test):      # query, 즉 미래 값 예측 함수
        z2 = np.dot(test, self.W2) + self.b2         # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                             # 은닉층의 출력
        z3 = np.dot(a2, self.W3) + self.b3         # 은닉층의 선형회귀 값
        a3 = sigmoid(z3)                             # 은닉층의 출력
        z4 = np.dot(a3, self.W4) + self.b4           # 출력층의 선형회귀 값
        y = a4 = sigmoid(z4)                         # 출력층의 출력
        if y > 0.5:
            result = 1  # True
        else:
            result = 0  # False
        return result, y

마찬가지로 층 구성 후, step을 55000까지 늘려 보았고, 손실값이 0.0129까지 나오는 것을 확인하였다.

In [18]:
# XOR Gate 객체 생성
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 0])

xor = LogicGate("XOR", xdata, tdata)
xor.train() 

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1], [0.7, 0.3], [0.6, 0.4], [0.5, 0.5], [0.4, 0.6], [0.3, 0.7] ])
for data in test_data:
    r, y = xor.predict(data)
    print(data, "-->", r, "[%.3f]"%y[0])

XOR object is created
Step = 0    	Error Val = 3.5851
Step = 2000 	Error Val = 2.7715
Step = 4000 	Error Val = 2.7711
Step = 6000 	Error Val = 2.7705
Step = 8000 	Error Val = 2.7698
Step = 10000	Error Val = 2.7687
Step = 12000	Error Val = 2.7670
Step = 14000	Error Val = 2.7642
Step = 16000	Error Val = 2.7592
Step = 18000	Error Val = 2.7487
Step = 20000	Error Val = 2.7204
Step = 22000	Error Val = 2.6144
Step = 24000	Error Val = 2.3091
Step = 26000	Error Val = 1.9981
Step = 28000	Error Val = 0.9958
Step = 30000	Error Val = 0.2367
Step = 32000	Error Val = 0.1092
Step = 34000	Error Val = 0.0680
Step = 36000	Error Val = 0.0486
Step = 38000	Error Val = 0.0375
Step = 40000	Error Val = 0.0305
Step = 42000	Error Val = 0.0256
Step = 44000	Error Val = 0.0220
Step = 46000	Error Val = 0.0193
Step = 48000	Error Val = 0.0172
Step = 50000	Error Val = 0.0154
Step = 52000	Error Val = 0.0140
Step = 54000	Error Val = 0.0129
Training time =  0:01:46.434442
[0. 0.] --> 0 [0.004]
[0. 1.] --> 1 [0.998]
[1. 0.

문제 1번과의 손실값 차이를 비교하니까, 층 하나를 더 넣으니 확실히 손실값이 더 줄어들었다.